### 2021 Games

Run this code to update the 2021 conference games and results.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pickle

In [3]:
teams = ["Bethel", "Goshen", "Grace", "HU", "IWU", "Marian", "MVNU", "SAU", "SFU", "Taylor"]
t_nums = [1629, 1678, 1679, 1688, 1694, 1717, 1736, 1780, 1805, 1784]

In [4]:
urls = ['http://www.dakstats.com/WebSync/Pages/Team/TeamSchedule.aspx?association=10&sg=MBA&sea=NAIMBA_2021&team=' +
        str(num) for num in t_nums]
#Create a handle, page, to handle the contents of the website
pages = [requests.get(url) for url in urls]
#Store the page as an element tree using BeautifulSoup4
soups = [BeautifulSoup(page.content) for page in pages]
team_tables = [
  [
    [
      [td.get_text(strip=True) for td in tr.find_all('td')] 
      for tr in table.find_all('tr') 
    ]#for each row in each table
    for table in soup.find_all('table') 
  ]#for each table on each webpage
  for soup in soups 
]#for each team's webpage
headers = [['Date', 'Opponent', 'Location', 'Score', 'Outcome'] for tables in team_tables]
team_rows = [[r[:5] for r in tables[35][1::2]] for tables in team_tables]
dfc = [pd.DataFrame(columns = headers[i], data = team_rows[i]) for i in range(len(headers))]
conf_df = [df[df.Opponent.str.contains("*", regex = False)] for df in dfc]
conf_df_played = [df[df.Score.str.contains("-", regex = False)] for df in conf_df]
tidy_conf2021 = conf_df_played.copy()
for i,df in enumerate(tidy_conf2021):
    split_scores = df['Score'].str.replace(r"\(.*\)","").str.split('-', expand = True)
    tidy_conf2021[i] = df.assign(Score = pd.to_numeric(split_scores[0]),
                            Opp_score = pd.to_numeric(split_scores[1]),
                            Opponent = df.Opponent.str.replace(' \*', '', regex= True),
                            Date = pd.to_datetime(df.Date)
                            )
for i in range(len(teams)): #add column for team
    tidy_conf2021[i]["Team"] = teams[i]
    team = tidy_conf2021[i].pop("Team")
    tidy_conf2021[i].insert(1, team.name, team) #move team column to second

In [9]:
conf_w2021 = [df[df.Outcome.str.contains("W", regex = False)] for df in tidy_conf2021]
conf_w2021[2][-3:]

,Date,Team,Opponent,Location,Score,Outcome,Opp_score
6,2021-03-05,Grace,Bethel (Ind.),N,5,W,2
8,2021-03-06,Grace,Bethel (Ind.),H,8,W,0
9,2021-03-06,Grace,Bethel (Ind.),H,14,W,5


In [10]:
new_games = pd.concat(conf_w2021)
new_games = new_games.sort_values(['Date', 'Team'])
new_games.rename(columns={
    'Team': 'Win_Tm',
    'Opponent': 'Lose_Tm',
    'Score': 'W_Score',
    'Opp_score': 'L_Score'}, 
    inplace=True)
del new_games["Outcome"]

In [11]:
new_games.Lose_Tm.replace({
        'Bethel (Ind.)' : 'Bethel',
        'Taylor (Ind.)' : 'Taylor',
        'Spring Arbor (Mich.)' : 'SAU',
        'Huntington (Ind.)' : 'HU',
        'St. Francis (Ind.)' : 'SFU',
        'Indiana Wesleyan' : 'IWU',
        'Mount Vernon Nazarene (Ohio)' : 'MVNU',
        'Marian (Ind.)' : 'Marian',
        'Goshen (Ind.)' : 'Goshen',
        'Grace (Ind.)' : 'Grace'
        }, 
    inplace=True)

In [12]:
new_games[-5:]

,Date,Win_Tm,Lose_Tm,Location,W_Score,L_Score
15,2021-03-06,IWU,SFU,A,7,5
16,2021-03-06,IWU,SFU,A,11,4
15,2021-03-06,Marian,HU,N,9,3
19,2021-03-06,Taylor,Goshen,N,14,2
20,2021-03-06,Taylor,Goshen,N,6,1


In [13]:
with open('2021games.pkl', 'wb') as f:
    pickle.dump(new_games, f)